In [ ]:
import os
import numpy as np
from hostdesigner.host import Host
from hostdesigner.run import hd_run, initialize
from hostdesigner.hdo import Hdo
from hostdesigner.output import read_hdo, find_results
from hostdesigner.template import overlay_template
from moleidoscope.linker import Linker
from moleidoscope.visualize import show
from moleidoscope.geo.vector import find_closest

#### Generate overlay template

In [ ]:
linker_length = 5 # Length of linker (Angstroms)
linker_finder = overlay_template(linker_length, bond_dist=1.2)

#### Initialize run

In [ ]:
run_dir = os.path.join(os.getcwd(), 'over-d%i' % int(linker_length))
initialize(run_dir, linker_finder, run_type='OVERLAY')

#### Run HostDesigner

In [ ]:
hd_run(run_dir, verbose=True)

#### Visualize results

In [ ]:
results = find_results(run_dir)
hdo1 = read_hdo(results['hdo'][1]) # out_1.hdo -> sorted by RMSD
hdo2 = read_hdo(results['hdo'][0]) # out_2.hdo -> sorted by energy
hdo1.show(8, start=0, div=4, distance=(20, 20))

#### Sort linkers according to number of atoms

In [ ]:
hdo2 = hdo1.sort(var='n_atoms', table=False)
hdo2.show(8, start=0, div=4, distance=(20, 20))

#### Choose a linker

In [ ]:
linker = Host()
linker.read_hdo(hdo1.structures, idx=4)
linker.remove_atoms(['O'])
show(linker)

#### Add extra linker information

In [ ]:
connection_indices = [i for i, atom in enumerate(linker.atom_names) if atom == 'N']
idx1, idx2 = connection_indices
n1 = np.array(linker.atom_coors[idx1])
n2 = np.array(linker.atom_coors[idx2])
linker.remove_atoms(['N'])

c1 = np.array(find_closest(n1, linker.atom_coors))
c2 = np.array(find_closest(n2, linker.atom_coors))

link = Linker()
link.connections = [c1, c2]
link.vector = c2 - c1
link.length = np.linalg.norm(link.vector)
link.read_host(linker)

show(link)

In [ ]:
link.save()